# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [61]:
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace

#dependecies for compute cluster
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

#dataset dependencies
from azureml.core import Dataset
from sklearn.model_selection import train_test_split

#automl dependencies
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails

#dependecy for model deployment
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice, Webservice


In [4]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-137297
aml-quickstarts-137297
southcentralus
3d1a56d2-7c81-4118-9790-f85d1acf0c77


In [5]:
experiment_name = 'heart-failure-exp-sayed'
experiment=Experiment(ws, experiment_name)
project_folder = './heart-failure-project'
experiment

Name,Workspace,Report Page,Docs Page
heart-failure-exp-sayed,quick-starts-ws-137297,Link to Azure Machine Learning studio,Link to Documentation


## Compute cluster creation


In [12]:
# Choosing a name for the CPU cluster
amlcompute_cluster_name = "sayed-cluster2"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 3)


Creating
Succeeded................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [15]:
ds = Dataset.get_by_name(ws, name='heart-failure-sayed')
dataframe = ds.to_pandas_dataframe()
dataframe.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


In [19]:
#splitting datasets to test and training set
training_dataset, test_dataset = train_test_split(dataframe, test_size=0.2, random_state=111)

In [20]:
training_dataset

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
278,50.0,1,1051,1,30,0,232000.0,0.7,136,0,0,246,0
218,68.0,1,1021,1,35,0,271000.0,1.1,134,1,0,197,0
128,61.0,0,248,0,30,1,267000.0,0.7,136,1,1,104,0
35,69.0,0,582,1,35,0,228000.0,3.5,134,1,0,30,1
184,58.0,1,145,0,25,0,219000.0,1.2,137,1,1,170,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,65.0,1,113,1,60,1,203000.0,0.9,140,0,0,94,0
169,70.0,0,835,0,35,1,305000.0,0.8,133,0,0,145,0
275,45.0,0,582,0,38,1,422000.0,0.8,137,0,0,245,0
86,55.0,0,47,0,35,1,173000.0,1.1,137,1,0,79,0


In [22]:
test_y = training_dataset['DEATH_EVENT']
test_x = training_dataset.drop(['DEATH_EVENT'], axis=1)

In [29]:
os.makedirs('data', exist_ok=True)
training_data_path = 'data/trainingdata.csv'
training_dataset.to_csv(training_data_path)

datastore = ws.get_default_datastore()

datastore.upload(src_dir='data', target_path='data')

my_training_ds = Dataset.Tabular.from_delimited_files(datastore.path(training_data_path))

Uploading an estimated of 1 files
Uploading data/trainingdata.csv
Uploaded data/trainingdata.csv, 1 files out of an estimated total of 1
Uploaded 1 files


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [30]:
# automl settings
automl_settings = {
    "n_cross_validations": 5,
    "primary_metric": 'accuracy',
    "enable_early_stopping": True,
    "max_concurrent_iterations": 4,
    "max_cores_per_iteration": -1
}

# automl config
automl_config = AutoMLConfig(task = 'classification',
                            experiment_timeout_minutes=30,
                            compute_target = compute_target,
                            training_data = my_training_ds,
                            label_column_name = 'DEATH_EVENT',
                            featurization= 'auto',
                            path = project_folder,
                            debug_log = "automl_errors.log",
                            **automl_settings
                            )

In [31]:
# TODO: Submitting experiment
remote_run = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [34]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [43]:
best_run, fitted_model = remote_run.get_output()
best_run

Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


Experiment,Id,Type,Status,Details Page,Docs Page
heart-failure-exp-sayed,AutoML_ca19b72c-5a6a-4c23-a502-59e8d4f2094c_36,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [44]:
print(fitted_model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                    min_samples_split=0.10368421052631578,
                                                                                                    min_weight_fraction_leaf=0.0,
                                                                                                    n_estimators=25,
      

In [45]:
best_run_metrics = best_run.get_metrics()
best_run_metrics

{'precision_score_macro': 0.8473729471097892,
 'f1_score_micro': 0.861790780141844,
 'weighted_accuracy': 0.8929530140884822,
 'precision_score_micro': 0.861790780141844,
 'norm_macro_recall': 0.6287318976173466,
 'AUC_macro': 0.9011736275897398,
 'f1_score_weighted': 0.8593001982296216,
 'accuracy': 0.861790780141844,
 'matthews_correlation': 0.6594304794861755,
 'AUC_weighted': 0.9011736275897398,
 'AUC_micro': 0.9135314889215834,
 'balanced_accuracy': 0.8143659488086733,
 'log_loss': 0.3876625683843984,
 'recall_score_weighted': 0.861790780141844,
 'recall_score_macro': 0.8143659488086733,
 'average_precision_score_macro': 0.8691788718809359,
 'average_precision_score_micro': 0.9169739217642665,
 'average_precision_score_weighted': 0.9140136313900673,
 'recall_score_micro': 0.861790780141844,
 'f1_score_macro': 0.8215820072559066,
 'precision_score_weighted': 0.8720457510218148,
 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_ca19b72c-5a6a-4c23-a502-59e8d4f2094c_36/

In [77]:
#TODO: Save the best model

model_name = best_run.properties['model_name']
# register model in workspace
mymodel = remote_run.register_model(model_name=model_name, description='Best automml model heart failure model', tags=None)

In [78]:
mymodel

Model(workspace=Workspace.create(name='quick-starts-ws-137297', subscription_id='3d1a56d2-7c81-4118-9790-f85d1acf0c77', resource_group='aml-quickstarts-137297'), name=AutoMLca19b72c536, id=AutoMLca19b72c536:2, version=2, tags={}, properties={})

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [50]:
#Copy the already created scoring_file_v_1_0_0.py file from the automl run into the scope.py
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score.py')

In [54]:
inference_config = InferenceConfig(entry_script='score.py', environment=best_run.get_environment())

In [80]:
# deploye the model as a web service 
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, auth_enabled=True, enable_app_insights=True)

deployed_service = Model.deploy(workspace=ws, 
                       name='automl-best-model',
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       models=[mymodel])

In [81]:
deployed_service.wait_for_deployment(show_output = True)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running............................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [82]:
print(deployed_service.get_logs())

2021-02-05T11:40:27,220301543+00:00 - gunicorn/run 
2021-02-05T11:40:27,221579048+00:00 - iot-server/run 
2021-02-05T11:40:27,222109650+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2021-02-05T11:40:27,228259872+00:00 - rsyslog/run 
rsyslogd

TODO: In the cell below, send a request to the web service you deployed to test it.

In [89]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

data = {
    "data":
    [
        {
            'Column1': "0",
            'age': "75",
            'anaemia': "2",
            'creatinine_phosphokinase': "500",
            'diabetes': "1",
            'ejection_fraction': "50",
            'high_blood_pressure': "1",
            'platelets': "127000",
            'serum_creatinine': "130",
            'serum_sodium': "0",
            'sex': "1",
            'smoking': "1",
            'time': "4",
        }
    ],
}

body = str.encode(json.dumps(data))

url = 'http://152628f3-a8cb-4116-a091-3f7c2611d3cb.southcentralus.azurecontainer.io/score'
api_key = 'btWNaLY4LjCbuARcXgBrluF6bV2AAdGQ' # Replace this with the API key for the web service
headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(json.loads(error.read().decode("utf8", 'ignore')))

b'"{\\"result\\": [1]}"'


TODO: In the cell below, print the logs of the web service and delete the service

In [92]:
# delete service
deployed_service.delete()

No service with name automl-best-model found to delete.
